In [ ]:
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import wfdb
from wfdb import processing
import os
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
import scipy
import shutil
from IPython.display import display 


In [ ]:
dir1 = '/kaggle/input/bidmc-congestive-heart-failure/files/'
dir2 = '/kaggle/input/mitbih-normal-sinus-rhythm-database/mit-bih-normal-sinus-rhythm-database-1.0.0/'
df1, df2 = list(), list()

# all the header files from chf dataset
for i in os.listdir(dir1):
    if i.endswith(".hea"):
        df1.append(i)
# all the annotations from normal dataset
for i in os.listdir(dir2):
    if i.endswith(".atr"):
        df2.append(i)        

df1, df2 = pd.DataFrame(df1), pd.DataFrame(df2) 
df_1 = pd.read_csv("/kaggle/input/bidmc-congestive-heart-failure/files/RECORDS", header=None)
df_2 = pd.read_csv("/kaggle/input/mitbih-normal-sinus-rhythm-database/mit-bih-normal-sinus-rhythm-database-1.0.0/RECORDS", header=None)

In [ ]:
record = wfdb.rdrecord('/kaggle/input/bidmc-congestive-heart-failure/files/chf01', sampfrom=100, sampto=2100)
ann = wfdb.rdann('/kaggle/input/bidmc-congestive-heart-failure/files/chf01', 'ecg', sampfrom=100, sampto=2100)
wfdb.plot_wfdb(record, ann)
display(record.__dict__)

In [ ]:
plt.plot(record.p_signal)
print("Signal length: " + str(len(record.p_signal)))

In [ ]:
fs = 128
new_sig, new_ann = wfdb.processing.resample_multichan(record.p_signal, ann, record.fs, fs_target = fs)
record.fs = fs # Update record's fs
print("New frequency sample: " + str(record.fs))
plt.plot(new_sig)
print("New signal length: " + str(len(new_sig)))

In [ ]:
avg = new_sig.mean() # Count Mean once
sd = new_sig.std() # Count SD once
print("The average is " + str(avg) + "and the std is " + str(sd))
new_sig = (new_sig - avg)/(sd)
plt.plot(new_sig);

In [ ]:
# Update with pre-processed signal
record.p_signal = new_sig
record.sig_len = len(new_sig)
wfdb.plot_wfdb(record, new_ann)
display(record.__dict__)

In [ ]:
peak_indices = wfdb.processing.gqrs_detect(record.p_signal, record.fs)
# wfdb.processing.compute_hr(record.sig_len, peak_indices, record.fs)
print(len(peak_indices))
print(peak_indices)
print(peak_indices[0], peak_indices[1])
plt.plot(record.p_signal);
plt.plot(record.p_signal[(peak_indices[0]):(peak_indices[1])]);

In [ ]:
wfdb.io.show_ann_labels()
wfdb.io.show_ann_classes()